## building dataset

In [41]:
from torch.utils.data import Dataset
from torch import Tensor
class MyDataset(Dataset):
    def __getitem__(self, index):
        return Tensor([index, 0.]), Tensor([3. * index + 4])
    def __len__(self):
        return 400
training_dataset = MyDataset()
print(len(training_dataset))
print(training_dataset[0])
print(training_dataset[1])

400
(tensor([0., 0.]), tensor([4.]))
(tensor([1., 0.]), tensor([7.]))


## building dataloader

In [42]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_dataset, batch_size=4)
for x, y in train_dataloader:
    print(x)
    print(y)
    break

tensor([[0., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.]])
tensor([[ 4.],
        [ 7.],
        [10.],
        [13.]])


## building model

In [43]:
from torch import nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = nn.Linear(2, 1)
    def forward(self, x):
        return self.model(x)
model = MyModel()
print(model)

MyModel(
  (model): Linear(in_features=2, out_features=1, bias=True)
)


## building optimizer

In [44]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=1e-1)

## training

In [45]:
loss_fn = nn.MSELoss()
for epoch in range(100):
    for batch, (x, y) in enumerate(train_dataloader):
        loss = loss_fn(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 19:
        print(f'epoch: {epoch}, loss: {loss.item()}')
        for n, p in model.named_parameters():
            print(f'name, parameters: {n, p}')
        print()

epoch: 19, loss: 0.00132819265127182
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0008, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.7311], requires_grad=True))

epoch: 39, loss: 26.613737106323242
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0310, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([1.7559], requires_grad=True))

epoch: 59, loss: 0.004904955625534058
name, parameters: ('model.weight', Parameter containing:
tensor([[ 2.9938, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([6.5013], requires_grad=True))

epoch: 79, loss: 0.0010834857821464539
name, parameters: ('model.weight', Parameter containing:
tensor([[ 2.9926, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([6.9206], requires_grad=True))

epoch: 99, loss: 81.99793243408203
name, param

## testing

In [46]:
print(model(Tensor([100, 0])))
print(model(Tensor([10000000, 0])))

tensor([308.0248], grad_fn=<AddBackward0>)
tensor([30423264.], grad_fn=<AddBackward0>)


可以看到，这里泛化到`10000000`就有了不小的误差

## finetuning

In [47]:
optimizer = Adam(model.parameters(), lr=1e-4)
for epoch in range(500):
    for batch, (x, y) in enumerate(train_dataloader):
        loss = loss_fn(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 19:
        print(f'epoch: {epoch}, loss: {loss.item()}')
        for n, p in model.named_parameters():
            print(f'name, parameters: {n, p}')
        print()

epoch: 19, loss: 0.01000823825597763
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0009, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.7533], requires_grad=True))

epoch: 39, loss: 0.0021298080682754517
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0007, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.7682], requires_grad=True))

epoch: 59, loss: 3.5762786865234375e-07
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0005, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.8003], requires_grad=True))

epoch: 79, loss: 1.126900315284729e-05
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0004, -0.4809]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.8462], requires_grad=True))

epoch: 99, loss: 2.436339855194092e-06
na

## testing \# 2

In [48]:
print(model(Tensor([100, 0])))
print(model(Tensor([10000000, 0])))

tensor([304.0034], grad_fn=<AddBackward0>)
tensor([29999892.], grad_fn=<AddBackward0>)
